### Info
- Please **Run All Cells** as depicted below:
<img src="https://drive.google.com/uc?export=view&id=1ksyrthmzxFhkdOBokKKsZFyrD-3mlHKC" width="500">

- Click on the web app **URL** (at the bottom page) to open it in a new browser tab:
<img src="https://drive.google.com/uc?export=view&id=1NkVZHia8VyD6N08poJ4Fsop8Jbx_1mlL" width="500">

- **Troubleshoot**: please do a **Kernel Restart & Run All** as depicted below:
<img src="https://drive.google.com/uc?export=view&id=1vU1M9anhkp0lBrqbigMNL15Ek6bWPA20" width="500">

### State Trends

In [ ]:
from difflib import get_close_matches
import requests
import pandas as pd
import time

import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash_extensions import Download
from dash_extensions.snippets import send_data_frame
from jupyter_dash import JupyterDash
import plotly.express as px

#### Data Wrangling

In [ ]:
# per state time-series json API
url = "https://api.covid19india.org/v4/min/timeseries.min.json"
response_ts = requests.get(url)

In [ ]:
# read json and normalize
start_time = time.time()
wide_ts_df = pd.json_normalize(response_ts.json())
total_sec = time.time() - start_time
print(f"{round(total_sec,1)} secs execution")

# transform to long format
long_ts_df = wide_ts_df.columns.str.split(".", expand=True).droplevel(1).to_frame(
    index=False, name=["state", "time_period", "obs_type", "obs_cat"]
)
long_ts_df["val"] = wide_ts_df.values[0]

In [ ]:
# state codelist as of Covid19 India API
cl_states = {
    "AN": "ANDAMAN & NICOBAR ISLANDS",
    "AP": "ANDHRA PRADESH",
    "AR": "ARUNACHAL PRADESH",
    "AS": "ASSAM",
    "BR": "BIHAR",
    "CH": "CHANDIGARH",
    "CT": "CHHATTISGARH",
    "DL": "NCT OF DELHI",
    "DN": "DADRA, NAGAR HAVELI, DAMAN & DIU",
    "GA": "GOA",
    "GJ": "GUJARAT",
    "HP": "HIMACHAL PRADESH",
    "HR": "HARYANA",
    "JH": "JHARKHAND",
    "JK": "JAMMU & KASHMIR",
    "KA": "KARNATAKA",
    "KL": "KERALA",
    "LA": "LADHAK",
    "LD": "LAKSHADWEEP",
    "MH": "MAHARASHTRA",
    "ML": "MEGHALAYA",
    "MN": "MANIPUR",
    "MP": "MADHYA PRADESH",
    "MZ": "MIZORAM",
    "NL": "NAGALAND",
    "OR": "ODISHA",
    "PB": "PUNJAB",
    "PY": "PUDUCHERRY",
    "RJ": "RAJASTHAN",
    "SK": "SIKKIM",
    "TG": "TELANGANA",
    "TN": "TAMIL NADU",
    "TR": "TRIPURA",
    "TT": "Total India",
    "UP": "UTTAR PRADESH",
    "UT": "UTTARAKHAND",
    "WB": "WEST BENGAL",
}

In [ ]:
# geo-json file from user jbrobst
url_geo = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
response_geo = requests.get(url_geo)
geofile = response_geo.json()

In [ ]:
# state codelist from geo-json file
cl_states_geo = {
    'AN': 'Andaman & Nicobar',
    'AP': 'Andhra Pradesh',
    'AR': 'Arunachal Pradesh',
    'AS': 'Assam',
    'BR': 'Bihar',
    'CH': 'Chandigarh',
    'CT': 'Chhattisgarh',
    'DL': 'Delhi',
    'DN': 'Dadra and Nagar Haveli and Daman and Diu',
    'GA': 'Goa',
    'GJ': 'Gujarat',
    'HP': 'Himachal Pradesh',
    'HR': 'Haryana',
    'JH': 'Jharkhand',
    'JK': 'Jammu & Kashmir',
    'KA': 'Karnataka',
    'KL': 'Kerala',
    'LA': 'Ladakh',
    'LD': 'Lakshadweep',
    'MH': 'Maharashtra',
    'ML': 'Meghalaya',
    'MN': 'Manipur',
    'MP': 'Madhya Pradesh',
    'MZ': 'Mizoram',
    'NL': 'Nagaland',
    'OR': 'Odisha',
    'PB': 'Punjab',
    'PY': 'Puducherry',
    'RJ': 'Rajasthan',
    'SK': 'Sikkim',
    'TG': 'Telangana',
    'TN': 'Tamil Nadu',
    'TR': 'Tripura',
    'UP': 'Uttar Pradesh',
    'UT': 'Uttarakhand',
    'WB': 'West Bengal'
}

In [ ]:
# read states population from Excel File
excel_file = './population/Districtwise_Population_Projection_India_2021.xlsx'
states_sheet_df = pd.read_excel(
    excel_file, sheet_name='States', dtype=str, header=None
)

In [ ]:
# read state names (numbered id 1 to 37)
logic_id = (
    (
        pd.to_numeric(states_sheet_df.loc[:, 0], errors='coerce') >= 1
    ) & (
        pd.to_numeric(states_sheet_df.loc[:, 0], errors='coerce') <= 37
    )
)
excel_st_index = states_sheet_df[logic_id][0].index
excel_st_names = states_sheet_df.loc[excel_st_index, 1]
# match states names
cl_excel_st = {
    word: (
        get_close_matches(cl_states[word], excel_st_names, n=1, cutoff=0.55)[0]
        if word != "TT"
        else "INDIA"
    )
    for word in cl_states
}

In [ ]:
# get total population from column index
def get_excel_pop_from_col(col_ind):
    excel_pop_ICO = {
        code: states_sheet_df[states_sheet_df.loc[:, 1] == code_name][col_ind + 2]
        .apply(pd.to_numeric)
        .values[0][0]
        for code, code_name in cl_excel_st.items()
    }
    # rewrite 'DN' (sum of two reported states)
    logic_index = states_sheet_df[states_sheet_df.loc[:, 1] == cl_excel_st['DN']].index[0]
    excel_pop_ICO['DN'] = (
        states_sheet_df.loc[[logic_index - 1, logic_index], col_ind + 2]
        .apply(pd.to_numeric)
        .sum()
        .values[0]
    )
    return excel_pop_ICO

In [ ]:
# match column to read Population Proj 2021
possibilities = states_sheet_df.loc[0, :].fillna('')
column_name = get_close_matches('Population Projection 2021', possibilities, n=1, cutoff=0.95)[0]
col_index = possibilities[possibilities == column_name].index

In [ ]:
col_name_18 = get_close_matches('Projected Population 18+ (2021)', possibilities, n=1, cutoff=0.95)[0]
col_index_18 = possibilities[possibilities == col_name_18].index

In [ ]:
# create dataframe from dictioanry populations ICO
st_pop_ico_df = (
    pd.DataFrame.from_dict(get_excel_pop_from_col(col_index), orient='index')
    .reset_index()
    .rename(columns={"index": "state", 0: "population"})
)
st_pop_18_df = (
    pd.DataFrame.from_dict(get_excel_pop_from_col(col_index_18 - 2), orient='index')
    .reset_index()
    .rename(columns={"index": "state", 0: "population_18p"})
)

In [ ]:
# target states metadata
url = "https://api.covid19india.org/v4/min/data.min.json"
response_data = requests.get(url)

In [ ]:
# filter state metadata from json data
json_meta_st = {
    key_1: {
        key_2: response_data.json()[key_1][key_2]
        for key_2 in response_data.json()[key_1] if key_2 == 'meta'
    } for key_1 in response_data.json()
}

In [ ]:
# read json_meta_st and normalize
wide_meta_st_df = pd.json_normalize(json_meta_st, max_level=2)
# build temporary long format from column names
long_meta_st_df = wide_meta_st_df.columns.str.split(".", expand=True).droplevel(1).to_frame(
    index=False, name=["state", "column"]
)
long_meta_st_df["val"] = wide_meta_st_df.values[0]
# pivot temporary long into state metadata table
meta_st_df = long_meta_st_df.pivot(index='state', columns='column', values='val').reset_index()
# delete index name `column` from pivot
meta_st_df.rename_axis(None, axis=1, inplace=True)

# un nest state metadata tested column
tested_df = meta_st_df.tested.apply(pd.Series).rename(
    columns={"date": "test_date", "source": "test_source"}
)
# concat back to metadata
meta_st_df = pd.concat([meta_st_df, tested_df], axis = 1).drop('tested', axis = 1)

# un nest state metadata vaccinated column if present
if 'vaccinated' in meta_st_df.columns:
    vac_df = meta_st_df.vaccinated.apply(pd.Series).rename(
        columns={"date": "vaccinated_date", "source": "vaccinated_source"}
    )
    # concat back to metadata
    meta_st_df = pd.concat([meta_st_df, vac_df], axis = 1).drop('vaccinated', axis = 1)

# rename column population (to include ICO population)
meta_st_df.rename(columns={"population": "pop_API"}, inplace=True)
# left join data/meta
states_df = long_ts_df.merge(
    meta_st_df, on='state', how="left", sort=False
).merge(
    st_pop_ico_df, on='state', how="left", sort=False
).merge(
    st_pop_18_df, on='state', how="left", sort=False
)

In [ ]:
# under 20 India UNPD 2020
pop_Y0T19 = 487063.152e3
# share of pop Y0T19 India (state 'TT')
share_Y0T19 = (pop_Y0T19 / states_df[states_df.state == 'TT'].population.unique())[0]
# project states based on total population
states_df['draft_pop_20p'] = states_df.population * (1-share_Y0T19)

In [ ]:
# adjust population 18p if nan or zero (not yet ICO loaded)
logic_18p_nan_or_z = states_df.population_18p.isnull() | states_df.population_18p.eq(0)
# replace nan or zeros with draft_pop_20p
states_df.loc[logic_18p_nan_or_z, 'population_18p'] = states_df.loc[logic_18p_nan_or_z, 'draft_pop_20p']

In [ ]:
# trend sitan case incidence (last 7 days confirmed sum per million pop)
query_c = "obs_cat == 'confirmed' & obs_type == 'delta7'"
key_index = ['state', 'time_period']
case_inc = (
    states_df.query(query_c).set_index(key_index).val * 1e6 / states_df.query(query_c).set_index(key_index).population
).reset_index().rename(columns={0: "val"})
case_inc['obs_cat'] = 'CI'
case_inc['obs_type'] = 'delta7'

In [ ]:
# status sitan case incidence (previous week status)
today = pd.Timestamp.today()
prev_monday = (today - pd.to_timedelta(today.weekday(), unit='d')).strftime('%Y-%m-%d')
query_cw = "obs_cat == 'confirmed' & obs_type == 'delta7' & time_period == @prev_monday"
case_inc_w = (
    states_df.query(query_cw).set_index(key_index).val * 1e6 / states_df.query(query_cw).set_index(key_index).population
).reset_index().rename(columns={0: "val"})
case_inc_w['obs_cat'] = 'CIW'
case_inc_w['obs_type'] = 'delta7'

In [ ]:
# trend sitan test PR (last 7 days confirmed sum over tested sum)
query_t = "obs_cat == 'tested' & obs_type == 'delta7'"
tpr = (
    # assumes no delta7 zeros on denom
    states_df.query(query_c).set_index(key_index).val * 100 / states_df.query(query_t).set_index(key_index).val
).reset_index().rename(columns={0: "val"})
tpr['obs_cat'] = 'TPR'
tpr['obs_type'] = 'delta7'

In [ ]:
# status sitan test PR (previous week status)
query_tw = "obs_cat == 'tested' & obs_type == 'delta7' & time_period == @prev_monday"
tpr_w = (
    # assumes no delta7 zeros on denom
    states_df.query(query_cw).set_index(key_index).val * 100 / states_df.query(query_tw).set_index(key_index).val
).reset_index().rename(columns={0: "val"})
tpr_w['obs_cat'] = 'TPRW'
tpr_w['obs_type'] = 'delta7'

In [ ]:
# trend sitan Case Fatality Rate (last 7 days deceased sum over confirmed sum)
query_d = "obs_cat == 'deceased' & obs_type == 'delta7'"
cfr = (
    # assumes no delta7 zeros on denom
    states_df.query(query_d).set_index(key_index).val * 100 / states_df.query(query_c).set_index(key_index).val
).reset_index().rename(columns={0: "val"})
cfr['obs_cat'] = 'CFR'
cfr['obs_type'] = 'delta7'

In [ ]:
# status sitan Case Fatality Rate (previous week status)
query_dw = "obs_cat == 'deceased' & obs_type == 'delta7' & time_period == @prev_monday"
cfr_w = (
    # assumes no delta7 zeros on denom
    states_df.query(query_dw).set_index(key_index).val * 100 / states_df.query(query_cw).set_index(key_index).val
).reset_index().rename(columns={0: "val"})
cfr_w['obs_cat'] = 'CFRW'
cfr_w['obs_type'] = 'delta7'

In [ ]:
# trend sitan Vaccination Rate (total 1-dose vaccinated over drafted target population 20+)
query_v = "obs_cat == 'vaccinated1' & obs_type == 'total'"
vr = (
    states_df.query(query_v).set_index(key_index).val * 100 / states_df.query(query_v).set_index(key_index).population_18p
).reset_index().rename(columns={0: "val"})
vr['obs_cat'] = 'VR'
vr['obs_type'] = 'total'

In [ ]:
# status sitan Vaccination Rate (previous week status)
query_vw = "obs_cat == 'vaccinated1' & obs_type == 'total' & time_period == @prev_monday"
vr_w = (
    states_df.query(query_vw).set_index(key_index).val * 100 / states_df.query(query_vw).set_index(key_index).population_18p
).reset_index().rename(columns={0: "val"})
vr_w['obs_cat'] = 'VRW'
vr_w['obs_type'] = 'total'

In [ ]:
# status percent change in cases - PCC (previous week vs 2 previous weeks status)
monday_2w = (pd.to_datetime(prev_monday) - pd.to_timedelta(7, unit='d')).strftime('%Y-%m-%d')
query_c2w = "obs_cat == 'confirmed' & obs_type == 'delta7' & time_period == @monday_2w"
pcc = (
    (
        states_df.query(query_cw).set_index('state').val - states_df.query(query_c2w).set_index('state').val
    ) / states_df.query(query_cw).set_index('state').val * 100
).reset_index().rename(columns={0: "val"})
# assigned to previous monday
pcc['time_period'] = prev_monday
pcc['obs_cat'] = 'PCC'
pcc['obs_type'] = 'delta14_7'

In [ ]:
# concat sitans to states dataframe
states_df = pd.concat(
    [
        states_df,
        case_inc,
        tpr,
        cfr,
        vr,
        case_inc_w,
        tpr_w,
        cfr_w,
        vr_w,
        pcc,
    ],
    ignore_index=True
)

#### Data vis

In [ ]:
# detect proxy configuration for JupyterHub or Binder
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
# dropdowns: state, obs_type, obs_cat, time_period
cl_indicators = {
    'CI': '7D Case Incidence',
    'TPR': '7D Test Positivity Rate',
    'CFR': '7D Case Fatality Rate',
    'VR': 'Total Vaccination Rate',
}

cl_indicators_status = {
    'CIW': 'Previous Week Case Incidence',
    'PCC': 'Percent Change in Cases (Previous Week vs 2-Previous Weeks)',
    'TPRW': 'Previous Week Test Positivity Rate',
    'CFRW': 'Previous Week Case Fatality Rate',
    'VRW': 'Previous Week Total Vaccination Rate',
}

# sitan thresholds WHO
sitan_thresh = {
    'CIW': {'lower': 100, 'upper': 200},
    'PCC': {'lower': 0, 'upper': 50},
    'TPRW': {'lower': 1, 'upper': 2},
    'CFRW': {'lower': 'NA', 'upper': 'NA'},
    'VRW': {'lower': 50, 'upper': 75},
}

dd_st = dcc.Dropdown(
    id="my_st",
    options=[
        {"label": value, "value": key}
        for key, value in cl_states.items()
    ],
    value='DL'
)
dd_ind = dcc.Dropdown(
    id="my_ind",
    options=[
        {"label": value, "value": key}
        for key, value in cl_indicators.items()
    ],
    value='CI'
)
dd_ind2 = dcc.Dropdown(
    id="my_ind2",
    options=[
        {"label": value, "value": key}
        for key, value in cl_indicators_status.items()
    ],
    value='CIW'
)
time_ps = sorted(long_ts_df.time_period.unique(), reverse=True)
dd_time = dcc.Dropdown(
    id="my_time",
    options=[
        {"label": value, "value": key}
        for key, value in zip(time_ps, time_ps)
    ],
    value='2021-04-01'
)
w1_input = dcc.Input(
    id = 'w1',
    value='0.25',
    type='number',
    debounce=True,
    min = 0,
    max = 1,
    step = 0.01,
    placeholder="[0 - 1]"
)
w2_input = dcc.Input(
    id = 'w2',
    value='0.25',
    type='number',
    debounce=True,
    min = 0,
    max = 1,
    step = 0.01,
    placeholder="[0 - 1]"
)
w3_input = dcc.Input(
    id = 'w3',
    value='0.25',
    type='number',
    debounce=True,
    min = 0,
    max = 1,
    step = 0.01,
    placeholder="[0 - 1]"
)
w4_input = dcc.Input(
    id = 'w4',
    value='0.25',
    type='number',
    debounce=True,
    min = 0,
    max = 1,
    step = 0.01,
    placeholder="[0 - 1]"
)

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# Build App
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
# App Layout
app.layout = html.Div([
    html.H3("Indian States Covid-19 Trends"),
    html.H6("Browse by State, SitAn Indicator and cut-off time"),
    html.Div([
        html.Div(
            ["Select State", dd_st],
            style={'width': '24%', 'display': 'inline-block'},
        ),
        html.Div(
            ["Select SitAn Indicator", dd_ind],
            style={'width': '24%', 'display': 'inline-block'},
        ),
        html.Div(
            ["Select cut-off time", dd_time],
            style={'width': '24%', 'display': 'inline-block'},
        ),
    ]),
    dcc.Graph(id='time-series'),
    html.H3("Indian States SitAn Weekly Status"),
    html.Div([
        html.Div(
            ["Select SitAn Indicator", dd_ind2],
            style={'width': '45%', 'display': 'inline-block'},
        ),
    ]),
    dcc.Graph(id='bar-plot'),
    # hidden div to share data across
    html.Div(id='store-df', style={'display': 'none'}),
    html.Div(
        [
            html.Button("Download SitAn Indicator", id="btn"),
            Download(id="download"),
        ]
    ),
    dcc.Graph(id='country-plot'),
    html.H3("Global SitAn: test weights"),
    html.Div([
        html.Div(
            [html.Label("w1 - Case Incidence"), w1_input],
            style={'width': '20%', 'display': 'inline-block'},
        ),
        html.Div(
            [html.Label("w2 - Change in Cases"), w2_input],
            style={'width': '20%', 'display': 'inline-block'},
        ),
        html.Div(
            [html.Label("w3 - Test Positivity Rate"), w3_input],
            style={'width': '20%', 'display': 'inline-block'},
        ),
        html.Div(
            [html.Label("w4 - Total Vaccination Rate"), w4_input],
            style={'width': '20%', 'display': 'inline-block'},
        ),
        html.Div(
            html.Button("CALCULATE", id="btn_cal"),
            style={'width': '15%', 'display': 'inline-block'},
        )
    ]),
    dcc.Graph(id='country-test-plot'),
])

In [ ]:
# Define callback to update trends
@app.callback(
    Output("time-series", "figure"),
    Input("my_st", "value"),
    Input("my_ind", "value"),
    Input("my_time", "value"),
)
def query_2_plot(state, indicator, co_time):
    # return all times if co_time None
    co_time = co_time if co_time else states_df.time_period.min()
    # don't return plot if missing values for query
    if any([not state, not indicator]):
        return {}
    else:
        # assign category and type according to indicator
        obs_type = 'total' if indicator == 'VR' else 'delta7'
        query = "state == @state & obs_type == @obs_type & obs_cat == @indicator & time_period > @co_time"
        fig = px.line(
            states_df.query(query),
            x="time_period",
            y="val",
            line_shape="spline",
        ).update_traces(mode="lines+markers")
        return fig

In [ ]:
# define color function
# TODO: check impact of mapping NaN / missing states calculation (use of series index)!
def get_color(v, lower, upper, ind):
    if ind != 'VRW':
        if v <= lower:
            return 'green'
        elif v >= upper:
            return 'red'
        else:
            return 'orange'
    else:
        if v >= upper:
            return 'green'
        elif v <= lower:
            return 'red'
        else:
            return 'orange'

# Define callback to update bar-plot
@app.callback(
    Output("bar-plot", "figure"),
    Output("store-df", "children"),
    Input("my_ind2", "value"),
)
def plot_indicator(indicator):
    # don't return plot if missing values
    if not indicator:
        return {}
    else:
        # bring indicator
        query = "obs_cat == @indicator"
        ind_calc = states_df.query(query).set_index(
            'state'
        ).val.reset_index().rename(columns={0: "val"})
        
        # TODO: two alternatives here --> either map (eg: 1, 2, 3) or just color using WHO status
        # Done: color by bins
        low_limit = sitan_thresh[indicator]['lower']
        upp_limit = sitan_thresh[indicator]['upper']
        
        if indicator != "CFRW":
            # assign color
            colors = [get_color(v, low_limit, upp_limit, indicator) for v in ind_calc.val]
            # add color column to dataframe
            ind_calc['color'] = colors
        
        # order of values for status
        order_val = 'total ascending' if indicator == 'VRW' else 'total descending'
        
        fig = px.bar(
                ind_calc,
                x='state',
                y="val",
                color=colors if "CFRW" not in indicator else None,
                color_discrete_map={
                    'red': 'red',
                    'orange': 'orange',
                    'green': 'green'
                },
            ).update_layout(
            xaxis={'categoryorder':order_val},
        )
        
        # share date to download
        data_dwnld = ind_calc.to_json(orient='split')
        
        return fig, (data_dwnld, indicator)

In [ ]:
# callback to show country plot (execute after selection of indicator dropdown)
@app.callback(
    Output("country-plot", "figure"),
    Input("store-df", "children"),
)
def plot_map(sitan_stored):
    indicator = sitan_stored[1]
    if indicator == "CFRW":
        # scale not yet defined for fatality rate
        return {}
    else:
        sitan_ind = pd.read_json(sitan_stored[0], orient='split')
        # drop total india --> code 'TT' and unknown 'UN'
        filter_tot_india = (sitan_ind.state == 'TT') | (sitan_ind.state == 'UN')
        sitan_ind.drop(sitan_ind[filter_tot_india].index, inplace=True)
        # add label for state codes
        state_labels = [cl_states_geo[c] for c in sitan_ind.state]
        # add state labels to dataframe
        sitan_ind['st_label'] = state_labels
        fig = px.choropleth(
            sitan_ind,
            geojson=geofile,
            featureidkey='properties.ST_NM',
            locations='st_label',
            color='color',
            color_discrete_map={
                'red':'red',
                'orange':'orange',
                'green':'green'
            },
            projection='mercator',
            hover_data=['val'],
        )
        fig.update_geos(fitbounds='locations', visible=False)
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        return fig

In [ ]:
@app.callback(
    Output("country-test-plot", "figure"),
    Input("btn_cal", "n_clicks"),
    State("w1", "value"),
    State("w2", "value"),
    State("w3", "value"),
    State("w4", "value"),
)
def calc_glob_sitan(n_clicks, w1, w2, w3, w4):
    
    # scale sitan except CFRW: fatality rate threshold NA
    sitan_dict_df = {
        ind: (
            states_df.query("obs_cat == @ind").set_index('state')
            .val.reset_index().rename(columns={0: "val"})
        )
        for ind in cl_indicators_status
        if ind != 'CFRW'
    }
    sitan_scales = {
        ind: (
            sitan_dict_df[ind]
            .set_index("state").val
            .apply(
                get_color,
                lower=sitan_thresh[ind]["lower"],
                upper=sitan_thresh[ind]["upper"],
                ind=ind,
            )
            .map({'green': 1, 'orange': 2, 'red': 3})
        )
        for ind in sitan_dict_df
    }
    
    # sitan global (research more elegant weighted sum)
    sitan_glob = (
        (
            float(w1) * sitan_scales["CIW"]
            + float(w2) * sitan_scales["PCC"]
            + float(w3) * sitan_scales["TPRW"]
            + float(w4) * sitan_scales["VRW"]
        )
        .apply(lambda x: int(x + 0.5))
        .map({1: "green", 2: "orange", 3: "red"})
        .reset_index()
    )
    
    # drop total india --> code 'TT' and unknown 'UN'
    filter_tot_or_un = (sitan_glob.state == 'TT') | (sitan_glob.state == 'UN')
    sitan_glob.drop(sitan_glob[filter_tot_or_un].index, inplace=True)
    # add label for state codes
    state_labels = [cl_states_geo[c] for c in sitan_glob.state]
    # add state labels to dataframe
    sitan_glob['st_label'] = state_labels
    
    # global sitan states map
    fig = px.choropleth(
        sitan_glob,
        geojson=geofile,
        featureidkey='properties.ST_NM',
        locations='st_label',
        color='val',
        color_discrete_map={
            'red':'red',
            'orange':'orange',
            'green':'green'
        },
        projection='mercator',
    )
    fig.update_geos(fitbounds='locations', visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    return fig

In [ ]:
@app.callback(
    Output("download", "data"),
    Input("btn", "n_clicks"),
    State("store-df", "children"),
    prevent_initial_call=True,
)
def func(n_clicks, data_dwnld):
    stored_data = pd.read_json(data_dwnld[0], orient='split')
    return send_data_frame(stored_data.to_csv, f"{data_dwnld[1]}.csv", index=False)

#### Situation Analysis Indicators:

|  |  |
| --- | --- |
| <img src="https://drive.google.com/uc?export=view&id=1X1hVR5y00vprU1jFT20nSP3Jc41jVsWY" width="200"> | <img src="https://drive.google.com/uc?export=view&id=1saMjeevjiVlv_Dq7BNRNUgKdOApjwFeS" width="200"> |
| <img src="https://drive.google.com/uc?export=view&id=10frXzVNHFAFNW1GrErj3QwKZRZGPZl9A" width="200"> | <img src="https://drive.google.com/uc?export=view&id=1AdDqL3kVyjaepYR8N9t6Q5Y2iwaXCkYK" width="200"> |
| <img src="https://drive.google.com/uc?export=view&id=1dg3qZfbjQFuxCyLsmTS6iLLD2BKT5yL2" width="200"> |  |

In [ ]:
# Run app and print out the application URL
app.run_server(mode='external')